### Installing Libraries for LangChain and Supporting Tools


In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pylcs
!pip3 install pypdf

### Importing Necessary Libraries


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import  PyPDFLoader
from langchain.vectorstores import  FAISS
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings 
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from time import monotonic


import textwrap
import os
import ast
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_similarity
)

from helper_functions import num_tokens_from_string, replace_t_with_space, replace_double_lines_with_one_line, split_into_chapters, is_similarity_ratio_lower_than_th, analyse_metric_results

### Setting Preferred Encoding for PyPDF on Google Colab


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding # For using PyPDF on google colab 

### Setting OpenAI API Key


In [ ]:
# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")
# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key
print("OPENAI_API_KEY has been set!")

### Defining Path to Harry Potter PDF


In [ ]:
hp_pdf_path ="Harry_Potter_Book_1_The_Sorcerers_Stone.pdf"

### Splitting the PDF into Chapters and Preprocessing


In [ ]:
chapters = split_into_chapters(hp_pdf_path) 
chapters = replace_t_with_space(chapters)
print(len(chapters))

### Defining Prompt Template for Summarization


In [ ]:
summarization_prompt_template = """Write an extensive summary of about of the following:

{text}

SUMMARY:"""

summarization_prompt = PromptTemplate(template=summarization_prompt_template, input_variables=["text"])

### Defining Function to Create Chapter Summaries using LLMs


In [ ]:
def create_chapter_summary(chapter):
    """
    Creates a summary of a chapter using a large language model (LLM).

    Args:
        chapter: A Document object representing the chapter to summarize.

    Returns:
        A Document object containing the summary of the chapter.
    """

    chapter_txt = chapter.page_content  # Extract chapter text
    model_name = "gpt-3.5-turbo-0125"  # Specify LLM model
    llm = ChatOpenAI(temperature=0, model_name=model_name)  # Create LLM instance
    gpt_35_turbo_max_tokens = 16000  # Maximum token limit for the LLM
    verbose = False  # Set to True for more detailed output

    # Calculate number of tokens in the chapter text
    num_tokens = num_tokens_from_string(chapter_txt, model_name)

    # Choose appropriate chain type based on token count
    if num_tokens < gpt_35_turbo_max_tokens:
        chain = load_summarize_chain(llm, chain_type="stuff", prompt=summarization_prompt, verbose=verbose)
    else:
        chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=summarization_prompt, combine_prompt=summarization_prompt, verbose=verbose)

    start_time = monotonic()  # Start timer
    doc_chapter = Document(page_content=chapter_txt)  # Create Document object for chapter
    summary = chain.invoke([doc_chapter])  # Generate summary using the chain
    print(f"Chain type: {chain.__class__.__name__}")  # Print chain type
    print(f"Run time: {monotonic() - start_time}")  # Print execution time

    # Clean up summary text
    summary = replace_double_lines_with_one_line(summary["output_text"])

    # Create Document object for summary
    doc_summary = Document(page_content=summary, metadata=chapter.metadata)

    return doc_summary

### Generating Summaries for Each Chapter


In [ ]:
chapter_summaries = []
for chapter in chapters:
    chapter_summaries.append(create_chapter_summary(chapter))

### Function to Encode a Book into a Vector Store using OpenAI Embeddings


In [ ]:
def encode_book(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

### Encoding Chapter Summaries into Vector Store


In [ ]:
def encode_chapter_summaries(chapter_summaries):
    """
    Encodes a list of chapter summaries into a vector store using OpenAI embeddings.

    Args:
        chapter_summaries: A list of Document objects representing the chapter summaries.

    Returns:
        A FAISS vector store containing the encoded chapter summaries.
    """

    embeddings = OpenAIEmbeddings()  # Create OpenAI embeddings
    chapter_summaries_vectorstore = FAISS.from_documents(chapter_summaries, embeddings)  # Create vector store
    return chapter_summaries_vectorstore

### Creating Vector Stores and Retrievers for Book and Chapter Summaries


In [ ]:
# Encode the book and chapter summaries into vector stores
chunks_vector_store = encode_book(hp_pdf_path, chunk_size=1000, chunk_overlap=200)
chapter_summaries_vector_store = encode_chapter_summaries(chapter_summaries)

# Create retrievers for both vector stores
chunks_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 4})
chapter_summaries_retriever = chapter_summaries_vector_store.as_retriever(search_kwargs={"k": 4})

### Generating Sub-Questions for Vector Store Queries from User Questions


In [ ]:
modifier_prompt_template = """
You are an expert on analyzing and understanding books based solely on their textual content. You have no prior knowledge about the specific book in the question.
If you ever read the related book before, FORGET ANYTHING about it.
You have access to two vector stores:

• One containing all the book content divided into chunks of 1000 characters with 200 character overlap.
• Another containing summaries of each chapter, approximately 250 tokens each.

Given a user's question about the book, your task is to generate a list of no more than 3 sub-questions that can be used as queries to retrieve relevant information from the vector stores based on semantic similarity. These sub-questions should be designed to collectively cover all the information needed to answer the original question comprehensively, without relying on any prior knowledge of the book's plot, characters, or events.

When generating the sub-questions, consider the following:


1.No Pre-knowledge: you're unaware of the book's details like plot or characters. The sub-questions must not present any prior knowledge of the book.
2.Directly Derived: Create sub-questions strictly from the user's question, aiming to retrieve book information without presupposing specific plot details or events.
3.Break Down: Decompose the user's question into finer, detailed sub-questions using only the textual content provided.
4.Key Concepts: Identify essential information needed from the original question and form targeted sub-questions to gather this data.
5.Self-contained Queries: Each sub-question should stand alone for effective vector store querying through semantic similarity.
6.Logical Sequence: Arrange sub-questions in a logical order that collectively provides a thorough answer.
7.Efficiency: Ensure sub-questions are unique and focused to avoid redundant searches and streamline information retrieval.

Output your response as a Python list, where each item is a self-contained sub-question that can be used as a standalone query for vector retrieval.

User's question: {question}
"""
modifier_prompt = PromptTemplate(
input_variables=["question"],
template=modifier_prompt_template,
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview", max_tokens=4000)

question_modifier_llm_chain = LLMChain(llm=llm, prompt=modifier_prompt)

### Modifying and Cleaning User Questions for Enhanced Querying


In [ ]:
def modify_question(question):
    """
    Modifies a question using a question modifier LLM chain and cleans up the output.

    Args:
        question: The input question string.

    Returns:
        A list of modified questions, or an empty list if there's an error.
    """

    # Invoke the question modifier LLM chain
    result = question_modifier_llm_chain.invoke(input=question)
    modified_questions_str = result['text']

    # Clean up the output string
    clean_str = modified_questions_str.replace("```python", "").replace("```", "").strip()  # Remove Markdown code block syntax
    clean_str = clean_str.replace("sub_questions = ", "").strip()  # Remove variable assignment
    clean_str = textwrap.dedent(clean_str)  # Dedent to remove unexpected indents

    # Attempt to convert the cleaned string into a list of questions
    try:
        modified_questions = ast.literal_eval(clean_str)
    except SyntaxError as e:
        print(f"Syntax error during ast.literal_eval: {e}")
        modified_questions = []  # Default to an empty list in case of error

    return modified_questions

### Example: Generating Modified Questions from an Original Query


In [ ]:
question = "how did harry beat quirrell?"
modified_question = modify_question(question)
print(modified_question) # watch new questions generated based on the original question

### Creating Context for a Question by Retrieving Relevant Documents and Summaries


In [ ]:
def create_context_per_question(question, multi_query_retriever, multi_query_retriever_chapter_summaries):
    """
    Creates context for a question by retrieving relevant documents and chapter summaries.

    Args:
        question: The input question string.
        multi_query_retriever: A retriever for retrieving relevant documents.
        multi_query_retriever_chapter_summaries: A retriever for retrieving relevant chapter summaries.

    Returns:
        A tuple containing two strings:
            - context: The concatenated content of relevant documents.
            - context_summaries: The concatenated content of relevant chapter summaries with citation information.
    """

    # Retrieve relevant documents and chapter summaries
    docs = multi_query_retriever.get_relevant_documents(question)
    docs_summaries = multi_query_retriever_chapter_summaries.get_relevant_documents(question)

    # Concatenate document content
    context = " ".join(doc.page_content for doc in docs)

    # Concatenate chapter summaries with citation information
    context_summaries = "".join(
        f"{doc.page_content} (Chapter {doc.metadata['chapter']})" for doc in docs_summaries
    )

    return context, context_summaries

### Comprehensive Question Answering Pipeline Using Context Retrieval and LLM


In [ ]:
def answer_question_pipeline(question, chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm, similarity_th=0.5):
    """
    Answers a question by retrieving relevant context, modifying the question, and using an LLM chain.

    Args:
        question: The input question string.
        retriever: A retriever for retrieving relevant documents.
        chapter_summaries_retriever: A retriever for retrieving relevant chapter summaries.
        answer_from_context_llm_chain: An LLM chain for answering questions based on context.
        multi_query_retriver_llm: An LLM for use in the MultiQueryRetriever.
        similarity_th:  Similarity threshold for context accumulation

    Returns:
        A tuple containing:
            - result: The result of invoking the answer_from_context_llm_chain.
            - all_context_book: The concatenated content of relevant documents.
            - all_context_summaries: The concatenated content of relevant chapter summaries.
    """

    # Create MultiQueryRetrievers
    multi_query_retriever = MultiQueryRetriever.from_llm(retriever=chunks_retriever, llm=multi_query_retriver_llm)
    multi_query_retriever_chapter_summaries = MultiQueryRetriever.from_llm(retriever=chapter_summaries_retriever, llm=multi_query_retriver_llm)

    # Modify the question
    modified_questions = modify_question(question)

    # Accumulate relevant context
    all_context_book = ""
    all_context_summaries = ""

    for modified_question in modified_questions:
        curr_question_relevant_context, curr_question_relevant_summaries_context = create_context_per_question(
            modified_question, multi_query_retriever, multi_query_retriever_chapter_summaries
        )

        # Add context if it's not too similar to existing context
        if is_similarity_ratio_lower_than_th(all_context_book, curr_question_relevant_context, similarity_th):
            all_context_book += curr_question_relevant_context

        if is_similarity_ratio_lower_than_th(all_context_summaries, curr_question_relevant_summaries_context, similarity_th):
            all_context_summaries += curr_question_relevant_summaries_context

    # Combine context from book and summaries
    all_context = all_context_book + all_context_summaries

    # Prepare input data for the LLM chain
    input_data = {
        "context": all_context,
        "question": question
    }

    # Execute the LLM chain and get the response
    result = answer_from_context_llm_chain.invoke(input=input_data)

    return result, all_context_book, all_context_summaries


### Template for Answering Questions Using Context-Specific Information


In [ ]:
answer_question_prompt_template = """
Based solely on the information provided in this context, and without using any information outside of this context, please answer the following question as concisely and as shortly as possible. You can rephrase the question for better fitting to the context.

Context:{context}
Question:{question}

**If the answer cannot be derived from the context, or if it requires knowledge from outside sources, simply answer: "I don't know".**

Please cite specific parts of the context in your answer to demonstrate how it supports your response.
If the chapter number of the relevant context appears, specify it in your answer.
"""
answer_question_prompt = PromptTemplate(
input_variables=["context", "question"],
template=answer_question_prompt_template,
)

multi_query_retriver_llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview", max_tokens=4000)

answer_from_context_llm_chain = LLMChain(llm=llm, prompt=answer_question_prompt)

### Example Execution of the Question Answering Pipeline for "How Did Harry Beat Quirrell?"


In [ ]:
question = 'how did harry beat quirrell?'
result, all_context_book, all_context_summaries = answer_question_pipeline(question,chunks_retriever,chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm)

wrapped_all_context_book = textwrap.fill(all_context_book, width=120)
print(f' conetxt book: {wrapped_all_context_book} \n')

wrapped_all_context_summaries = textwrap.fill(all_context_summaries, width=120)
print(f' context summaries: {wrapped_all_context_summaries} \n')

wrapped_result = textwrap.fill(result['text'], width=120)
print(f' answer: {wrapped_result}')

### Model Evaluation


In [ ]:
questions = [
    "Who gave Harry Potter his first broomstick?",
    "What is the name of the three-headed dog guarding the Sorcerer's Stone?",
    "Which house did the Sorting Hat initially consider for Harry?",
    "What is the name of Harry's owl?"
]
#     "How did Harry and his friends get past Fluffy?",
#     "What is the Mirror of Erised?",
#     "Who tried to steal the Sorcerer's Stone?",
#     "How did Harry defeat Quirrell/Voldemort?",
#     "What is Harry's parent's secret weapon against Voldemort?",
# ]

ground_truth_answers = [
    "Professor McGonagall",
    "Fluffy",
    "Slytherin",
    "Hedwig",
    # "They played music to put Fluffy to sleep.",
    # "A magical mirror that shows the 'deepest, most desperate desire of our hearts.'",
    # "Professor Quirrell, possessed by Voldemort",
    # "Harry's mother's love protected him, causing Quirrell/Voldemort pain when they touched him.",
    # "Love",
]

### Generating Answers and Retrieving Documents for Predefined Questions


In [ ]:
generated_answers = []
retrieved_documents = []
for question in questions:
    result, all_context_book, all_context_summaries = answer_question_pipeline(question, chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm)
    generated_answers.append(result['text'])
    retrieved_documents.append(all_context_book + all_context_summaries)


### Displaying Retrieved Documents and Generated Answers


In [ ]:
print(f'retrieved_documents: {retrieved_documents}\n')
print(f'generated_answers: {generated_answers}')

### Preparing Data and Conducting Ragas Evaluation


In [ ]:
# Prepare data for Ragas evaluation
data_samples = {
    'question': questions,  # Replace with your list of questions
    'answer': generated_answers,  # Replace with your list of generated answers
    'contexts': retrieved_documents,  # Your retrieved_documents list
    'ground_truth': ground_truth_answers  # Replace with your list of ground truth answers
}

# Convert contexts to list of strings (if necessary)
data_samples['contexts'] = [list(context) for context in data_samples['contexts']]

dataset = Dataset.from_dict(data_samples)

# Evaluate using Ragas with the specified metrics
metrics = [
    answer_correctness,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_similarity
]
llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview", max_tokens=4000)
score = evaluate(dataset, metrics=metrics, llm=llm)

# Print results and explanations
results_df = score.to_pandas()
print(results_df)

### Analyzing Metric Results from Ragas Evaluation


In [ ]:
analyse_metric_results(results_df)

### Interactive Chat Interface for Harry Potter Inquiries


In [ ]:
def chat_with_data(chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm):
    """
    Provides an interactive chat interface for answering questions about Harry Potter.

    Args:
        retriever: A retriever for retrieving relevant documents.
        chapter_summaries_retriever: A retriever for retrieving relevant chapter summaries.
        answer_from_context_llm_chain: An LLM chain for answering questions based on context.
        multi_query_retriver_llm: An LLM for use in the MultiQueryRetriever.
    """

    print("You can start chatting with me about Harry Potter. Type 'exit' to stop.")

    while True:
        # Prompt the user for a question
        question = input("What's your question? \n")

        # Check if the user wants to exit
        if question.lower() == 'exit':
            print("Exiting chat. Goodbye!")
            break

        # Answer the question using the pipeline
        result, _, _ = answer_question_pipeline(
            question, chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm
        )

        # Print the answer
        print("Answer:")
        wrapped_result = textwrap.fill(result['text'], width=120)  # Wrap text for readability
        print(wrapped_result)
        print("-" * 80)  # Print a separator line for readability

### Calling the chat_with_data function

In [ ]:
chat_with_data(chunks_retriever,chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm)